<a href="https://colab.research.google.com/github/ZHTushar23/Neural-Networking/blob/master/Linear_regression_with_pytorch__built_ins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [0]:
import torch.nn as nn
import numpy as np
import torch

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

# Dataset and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [0]:
from torch.utils.data import TensorDataset


In [0]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[1:10]

(tensor([[ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]), tensor([[ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

The *TensorDataset* allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a **DataLoader**, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [0]:
from torch.utils.data import DataLoader

In [0]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a for-in loop. Let's look at an example.

In [0]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.]])
tensor([[ 81., 101.],
        [ 22.,  37.],
        [103., 119.],
        [ 22.,  37.],
        [ 56.,  70.]])


# nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [0]:
# Define model
model = nn.Linear(3, 2)   #(#attributes,#class)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5394, -0.0564,  0.3494],
        [-0.0644,  0.0648,  0.0060]], requires_grad=True)
Parameter containing:
tensor([0.3401, 0.3356], requires_grad=True)


In [0]:
# Generate predictions
preds = model(inputs)
preds

tensor([[50.9645,  0.2345],
        [66.8279,  0.5623],
        [59.9805,  3.7630],
        [65.8632, -3.2228],
        [56.6068,  2.5327],
        [50.9645,  0.2345],
        [66.8279,  0.5623],
        [59.9805,  3.7630],
        [65.8632, -3.2228],
        [56.6068,  2.5327],
        [50.9645,  0.2345],
        [66.8279,  0.5623],
        [59.9805,  3.7630],
        [65.8632, -3.2228],
        [56.6068,  2.5327]], grad_fn=<AddmmBackward>)

# Loss Function
Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [0]:
#import nn.Functional
import torch.nn.functional as F

The nn.functional package contains many useful loss functions and several other utilities.

In [0]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [0]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(5462.5483, grad_fn=<MseLossBackward>)


# Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD stands for stochastic gradient descent. It is called stochastic because samples are selected in batches (often with random shuffling) instead of as a single group.

In [0]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

# Train the model
We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function fit which trains the model for a given number of epochs.

In [0]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Let's train the model for 100 epochs.

In [0]:
fit(100, model, loss_fn, opt,train_dl)

Epoch [10/100], Loss: 21.4916
Epoch [20/100], Loss: 18.9856
Epoch [30/100], Loss: 13.0190
Epoch [40/100], Loss: 10.5928
Epoch [50/100], Loss: 6.7816
Epoch [60/100], Loss: 5.1416
Epoch [70/100], Loss: 8.7981
Epoch [80/100], Loss: 7.0220
Epoch [90/100], Loss: 4.1957
Epoch [100/100], Loss: 2.6735
